In [1]:
# import necessary libraries
import pandas as pd
import requests
from io import StringIO

# define function name and parameters
def get_airports_table_from_github(url):
    
    # necessary info to connect with sql server
    schema="project_gans"
    host="gans-project-db.cnxahfg4n8cc.eu-west-3.rds.amazonaws.com"
    user="admin"
    password="your_password"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    # make request to get info from github and save the response
    table_url = url
    response = requests.get(table_url)
    
    # if response is 200, save the response in .text format, otherwise print error
    if response.status_code == 200:
      table_data = response.text 
    else:
      print("Failed to retrieve the table data. Status code:", response.status_code)
    
    # convert the table data to csv format
    csv_io = StringIO(table_data) 
    # read the csv format table data
    df = pd.read_csv(csv_io) 
    # convert the table data to pandas dataframe
    airports_df = pd.DataFrame(df) 
    # drop duplicate rows
    airports_df = airports_df.drop_duplicates() 
    # drop duplicate rows based on icao column
    airports_df = airports_df.drop_duplicates('icao') 
    #  drop rows with missing values
    airports_df = airports_df.dropna() 
    # rename columns
    airports_df = airports_df.rename(columns = {'iata' : 'airport_iata', 'icao' : 'airport_icao', 
                                                'airport' : 'airport_name'})
    # reorder columns
    airports_df = airports_df[['airport_icao', 'airport_name', 'country_code', 'region_name', 'airport_iata', 
                               'latitude', 'longitude']]
   
    # use .to_sql to send dataframe to sql
    airports_df.to_sql('airports',         
              if_exists='append', 
              con=con, 
              index=False) 
              
    # return created dataframe
    return(airports_df)

In [3]:
airports_df = get_airports_table_from_github('https://raw.githubusercontent.com/ip2location/ip2location-iata-icao/master/iata-icao.csv')

In [4]:
airports_df

,airport_icao,airport_name,country_code,region_name,airport_iata,latitude,longitude
0,OMSJ,Sharjah International Airport,AE,Ash Shariqah,SHJ,25.3286,55.5172
1,OMAD,Al Bateen Executive Airport,AE,Abu Zaby,AZI,24.4283,54.4581
2,OMFJ,Fujairah International Airport,AE,Al Fujayrah,FJR,25.1122,56.3240
3,OMBY,Sir Bani Yas Airport,AE,Abu Zaby,XSB,24.2836,52.5803
4,OMRK,Ras Al Khaimah International Airport,AE,Ra's al Khaymah,RKT,25.6135,55.9388
...,...,...,...,...,...,...,...
8965,FVTL,Thornhill Air Base,ZW,Midlands,GWE,-19.4364,29.8619
8966,FVBU,Joshua Mqabuko Nkomo International Airport,ZW,Bulawayo,BUQ,-20.0174,28.6179
8967,FVCH,Chipinge Airport,ZW,Manicaland,CHJ,-20.2067,32.6283
8968,FVKB,Kariba Airport,ZW,Mashonaland West,KAB,-16.5198,28.8850
